# Hydra для управления конфигурациями ML-экспериментов

## Проблема масштабирования: От скриптов к модулям



В реальном ML-проекте мы редко останавливаемся на обучении одной модели. Обычно исследовательский процесс выглядит как перебор гипотез.

Представьте, что вам нужно протестировать:
* **3 архитектуры модели** (ResNet, VGG, EfficientNet)
* **2 способа обработки данных** (с аугментацией и без)
* **4 набора гиперпараметров** (разные Learning Rate и Optimizer)


Если вы используете обычные скрипты, любой эксперимент превращается в проблему дублирования кода.
Например, чтобы проверить все сочетания из нашего примера (3 модели × 2 датасета × 4 параметра), вам теоретически нужно **24 разных варианта кода**. Поддерживать такое количество скриптов невозможно.

*  *А если вы найдете ошибку в цикле обучения, то вам придется исправлять её в **каждом** из десятка файлов.*

Hydra предлагает принципиально другой(модульный) подход через **композицию конфигов**.

Вместо того чтобы менять код, вы разбиваете конфигурацию на независимые блоки (как кубики LEGO):
* 📄 `conf/model/resnet.yaml`
* 📄 `conf/model/vgg.yaml`
* 📄 `conf/dataset/augmented.yaml`

У вас остается **всего один** скрипт `train.py`, в который вы "вставляете" нужные блоки при запуске:

```bash
# Запуск одной и той же программы с разными "кубиками":

1. python train.py model=resnet dataset=clean

2. python train.py model=vgg dataset=augmented


В этом блокноте мы рассмотрим:
1. Базовое использование Hydra для управления параметрами
2. Переопределение параметров из командной строки
3. Композицию конфигов для модульной архитектуры
4. Автоматический grid search с multirun
5. Анализ результатов экспериментов
6. Интеграцию с другими инструментами

Полезные ссылки:
* [Официальная документация Hydra](https://hydra.cc/)
* [GitHub репозиторий](https://github.com/facebookresearch/hydra)
* [Примеры использования](https://hydra.cc/docs/tutorials/intro/)

 Установка:

In [ ]:
!pip install -q hydra-core torch torchvision omegaconf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.7 MB/s eta 0:00:00


# Базовый подход: параметры в коде

Сначала посмотрим на типичный код обучения модели с хардкод параметрами.

In [ ]:
%%writefile train_baseline.py
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Параметры захардкожены в коде
LEARNING_RATE = 0.001
BATCH_SIZE = 64
HIDDEN_SIZE = 128
DROPOUT = 0.5
EPOCHS = 3

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, HIDDEN_SIZE)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(DROPOUT)
        self.fc2 = nn.Linear(HIDDEN_SIZE, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print(f"Параметры: LR={LEARNING_RATE}, Hidden={HIDDEN_SIZE}, Batch={BATCH_SIZE}")

    for epoch in range(EPOCHS):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()

        accuracy = 100. * correct / len(test_dataset)
        print(f"Epoch {epoch+1}/{EPOCHS}: Accuracy = {accuracy:.2f}%")

    print(f"\nФинальная accuracy: {accuracy:.2f}%")

if __name__ == "__main__":
    train()

Writing train_baseline.py


In [ ]:
!python train_baseline.py

100% 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100% 28.9k/28.9k [00:00<00:00, 473kB/s]
100% 1.65M/1.65M [00:00<00:00, 4.51MB/s]
100% 4.54k/4.54k [00:00<00:00, 16.3MB/s]
Параметры: LR=0.001, Hidden=128, Batch=64
Epoch 1/3: Accuracy = 94.94%
Epoch 2/3: Accuracy = 96.20%
Epoch 3/3: Accuracy = 96.79%

Финальная accuracy: 96.79%


# Использование Hydra

Теперь перепишем тот же код с использованием Hydra.

In [ ]:
# Создаём директорию для конфигов
!mkdir -p conf

In [ ]:
%%writefile conf/config.yaml
# Параметры модели
model:
  hidden_size: 128
  dropout: 0.5

# Параметры обучения
training:
  learning_rate: 0.001
  batch_size: 64
  epochs: 3

# Общие настройки
seed: 42

Writing conf/config.yaml


In [ ]:
%%writefile train_hydra.py
import hydra
from omegaconf import DictConfig
from hydra.core.hydra_config import HydraConfig
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import yaml

class SimpleNN(nn.Module):
    def __init__(self, hidden_size, dropout):
        super().__init__()
        self.fc1 = nn.Linear(784, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

@hydra.main(version_base=None, config_path="conf", config_name="config")
def train(cfg: DictConfig):

    torch.manual_seed(cfg.seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=cfg.training.batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=cfg.training.batch_size, shuffle=False)

    model = SimpleNN(hidden_size=cfg.model.hidden_size, dropout=cfg.model.dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=cfg.training.learning_rate)

    # цикл обучения
    for epoch in range(cfg.training.epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
        accuracy = 100. * correct / len(test_dataset)
        print(f"Epoch {epoch+1}: Accuracy = {accuracy:.2f}%")

    # Сохраняем результаты
    results = {
        'accuracy': float(accuracy),
        'params': {
            'learning_rate': cfg.training.learning_rate,
            'hidden_size': cfg.model.hidden_size,
            'batch_size': cfg.training.batch_size,
            'dropout': cfg.model.dropout
        }
    }

    hydra_output_dir = HydraConfig.get().runtime.output_dir
    save_path = os.path.join(hydra_output_dir, 'results.yaml')

    with open(save_path, 'w') as f:
        yaml.dump(results, f)

    print(f"\nФинальная accuracy: {accuracy:.2f}%")
    print(f"Файл сохранен в: {save_path}")

if __name__ == "__main__":
    train()

Writing train_hydra.py


In [ ]:
!python train_hydra.py

Epoch 1: Accuracy = 95.18%
Epoch 2: Accuracy = 96.28%
Epoch 3: Accuracy = 96.58%

Финальная accuracy: 96.58%
Файл сохранен в: /content/outputs/2026-01-25/20-38-14/results.yaml


### Структура созданных директорий

Hydra автоматически создаёт организованную структуру:

```
outputs/
  2025-01-24/              # Дата запуска
    15-30-45/              # Время запуска
      .hydra/
        config.yaml        # Использованная конфигурация
        hydra.yaml         # Настройки Hydra
        overrides.yaml     # Переопределения параметров
      results.yaml         # Результаты эксперимента
      train_hydra.log      # Логи выполнения
```

Каждый запуск изолирован в отдельной директории, и конфигурация сохраняется автоматически.

In [ ]:
# Посмотрим на созданные файлы
!find outputs -type f -name '*.yaml' | head -10

outputs/2026-01-25/19-46-30/results.yaml
outputs/2026-01-25/19-46-30/.hydra/hydra.yaml
outputs/2026-01-25/19-46-30/.hydra/overrides.yaml
outputs/2026-01-25/19-46-30/.hydra/config.yaml


## Переопределение параметров

Параметры можно переопределять из командной строки без изменения YAML файлов.

Синтаксис: `python script.py parameter.name=new_value`

In [ ]:
# Изменяем learning rate
!python train_hydra.py training.learning_rate=0.01

Epoch 1: Accuracy = 92.21%
Epoch 2: Accuracy = 92.78%
Epoch 3: Accuracy = 93.06%

Финальная accuracy: 93.06%
Файл сохранен в: /content/outputs/2026-01-25/19-47-19/results.yaml


In [ ]:
# Изменяем размер скрытого слоя
!python train_hydra.py model.hidden_size=512

Epoch 1: Accuracy = 96.52%
Epoch 2: Accuracy = 97.28%
Epoch 3: Accuracy = 97.57%

Финальная accuracy: 97.57%
Файл сохранен в: /content/outputs/2026-01-25/19-48-06/results.yaml


In [ ]:
# Изменяем несколько параметров одновременно
!python train_hydra.py \
    model.hidden_size=256 \
    training.learning_rate=0.0005 \
    training.batch_size=128

Epoch 1: Accuracy = 93.95%
Epoch 2: Accuracy = 95.79%
Epoch 3: Accuracy = 96.62%

Финальная accuracy: 96.62%
Файл сохранен в: /content/outputs/2026-01-25/19-48-53/results.yaml



Такой подход позволяет тестировать различные конфигурации без изменения файлов. Особенно удобно при работе с системами оркестрации(например, для запуска задач на кластере) можно передавать параметры через командную строку, сохраняя код неизменным.

## Композиция конфигов

Hydra позволяет создавать модульные конфигурации и комбинировать их. Это полезно когда нужно экспериментировать с различными компонентами системы.

Например, в проекте может быть:
- Несколько архитектур моделей
- Несколько способов обработки данных
- Несколько наборов данных

Вместо создания отдельного конфига для каждой комбинации, создаются модульные конфиги, которые можно комбинировать.

In [ ]:
# Создаём модульную структуру
!mkdir -p conf/model
!mkdir -p conf/optimizer

In [ ]:
%%writefile conf/model/simple.yaml
name: "SimpleNN"
hidden_size: 128
dropout: 0.5

Writing conf/model/simple.yaml


In [ ]:
%%writefile conf/model/large.yaml
name: "LargeNN"
hidden_size: 512
dropout: 0.3

Writing conf/model/large.yaml


In [ ]:
%%writefile conf/optimizer/adam.yaml
name: "adam"
learning_rate: 0.001
weight_decay: 0.0

Writing conf/optimizer/adam.yaml


In [ ]:
%%writefile conf/optimizer/sgd.yaml
name: "sgd"
learning_rate: 0.01
momentum: 0.9
weight_decay: 0.0001

Writing conf/optimizer/sgd.yaml


In [ ]:
%%writefile conf/config_modular.yaml
defaults:
  - model: simple
  - optimizer: adam
  - _self_

training:
  batch_size: 64
  epochs: 3

seed: 42

Writing conf/config_modular.yaml


In [ ]:
%%writefile show_config.py
import hydra
from omegaconf import DictConfig, OmegaConf

@hydra.main(version_base=None, config_path="conf", config_name="config_modular")
def show(cfg: DictConfig):
    print("\nКомпозированная конфигурация:")
    print("="*70)
    print(OmegaConf.to_yaml(cfg))

if __name__ == "__main__":
    show()

Writing show_config.py


In [ ]:
# Конфигурация по умолчанию: simple + adam
!python show_config.py


Композированная конфигурация:
model:
  name: SimpleNN
  hidden_size: 128
  dropout: 0.5
optimizer:
  name: adam
  learning_rate: 0.001
  weight_decay: 0.0
training:
  batch_size: 64
  epochs: 3
seed: 42



In [ ]:
# Переключаем на large + sgd
!python show_config.py model=large optimizer=sgd


Композированная конфигурация:
model:
  name: LargeNN
  hidden_size: 512
  dropout: 0.3
optimizer:
  name: sgd
  learning_rate: 0.01
  momentum: 0.9
  weight_decay: 0.0001
training:
  batch_size: 64
  epochs: 3
seed: 42





Модульный подход позволяет:
- Легко комбинировать различные компоненты
- Переиспользовать настройки между экспериментами
- Избежать дублирования конфигураций
- Стандартизировать настройки в команде

Например, для 3 моделей, 3 оптимизаторов и 2 датасетов нужно создать только 3+3+2=8 модульных конфигов вместо 3×3×2=18 полных конфигураций.

## Grid Search с Multirun

Hydra поддерживает автоматический запуск множественных экспериментов через флаг `-m` (multirun). Это позволяет тестировать различные комбинации параметров одной командой.

In [ ]:
# Запускаем grid search: 2 hidden sizes × 3 learning rates = 6 экспериментов
!python train_hydra.py -m \
    model.hidden_size=128,256 \
    training.learning_rate=0.0001,0.001,0.01 \
    training.epochs=2

[2026-01-25 19:49:39,158][HYDRA] Launching 6 jobs locally
[2026-01-25 19:49:39,158][HYDRA] 	#0 : model.hidden_size=128 training.learning_rate=0.0001 training.epochs=2
Epoch 1: Accuracy = 90.92%
Epoch 2: Accuracy = 92.35%

Финальная accuracy: 92.35%
Файл сохранен в: /content/multirun/2026-01-25/19-49-38/0/results.yaml
[2026-01-25 19:50:07,836][HYDRA] 	#1 : model.hidden_size=128 training.learning_rate=0.001 training.epochs=2
Epoch 1: Accuracy = 95.18%
Epoch 2: Accuracy = 96.28%

Финальная accuracy: 96.28%
Файл сохранен в: /content/multirun/2026-01-25/19-49-38/1/results.yaml
[2026-01-25 19:50:36,030][HYDRA] 	#2 : model.hidden_size=128 training.learning_rate=0.01 training.epochs=2
Epoch 1: Accuracy = 92.21%
Epoch 2: Accuracy = 92.78%

Финальная accuracy: 92.78%
Файл сохранен в: /content/multirun/2026-01-25/19-49-38/2/results.yaml
[2026-01-25 19:51:04,402][HYDRA] 	#3 : model.hidden_size=256 training.learning_rate=0.0001 training.epochs=2
Epoch 1: Accuracy = 91.79%
Epoch 2: Accuracy = 93.22%

Результаты организуются следующим образом:

```
multirun/
  2024-01-24/
    16-00-00/
      0/  # hidden=128, lr=0.0001
        .hydra/config.yaml
        results.yaml
      1/  # hidden=128, lr=0.001
        .hydra/config.yaml
        results.yaml
      2/  # hidden=128, lr=0.01
      3/  # hidden=256, lr=0.0001
      4/  # hidden=256, lr=0.001
      5/  # hidden=256, lr=0.01
      multirun.yaml
```

Каждый эксперимент сохраняет свою конфигурацию, что позволяет точно воспроизвести любой из них.

## Анализ результатов

После запуска grid search нужно проанализировать результаты и найти лучшую конфигурацию.

In [ ]:
%%writefile analyze_results.py
import yaml
from pathlib import Path

def analyze():
    multirun_base = Path("/content/multirun") if Path("/content").exists() else Path("multirun")

    dates = sorted([d for d in multirun_base.iterdir() if d.is_dir()])
    if not dates:
        return

    latest_run = sorted(list(dates[-1].iterdir()))[-1]
    print(f"\nАнализ запуска: {latest_run}\n")

    results = []
    for exp_dir in sorted(latest_run.iterdir()):
        if exp_dir.is_dir() and exp_dir.name.replace('_', '').isdigit():
            config_file = exp_dir / ".hydra" / "config.yaml"
            results_file = exp_dir / "results.yaml"

            if config_file.exists() and results_file.exists():
                try:
                    with open(config_file) as f:
                        config = yaml.safe_load(f)
                    with open(results_file) as f:
                        metrics = yaml.safe_load(f)

                    results.append({
                        'exp': exp_dir.name,
                        'hidden': config['model']['hidden_size'],
                        'lr': config['training']['learning_rate'],
                        'batch': config['training']['batch_size'],
                        'acc': metrics['accuracy']
                    })
                except:
                    pass

    if not results:
        print("Результаты не найдены")
        return

    results.sort(key=lambda x: x['acc'], reverse=True)

    print("Топ-5 конфигураций:\n")
    print(f"{'Rank':<6} {'Exp':<6} {'Hidden':<8} {'LR':<10} {'Batch':<7} {'Accuracy':<10}")
    print("-" * 60)

    for i, r in enumerate(results[:5], 1):
        print(f"{i:<6} {r['exp']:<6} {r['hidden']:<8} {r['lr']:<10.6f} {r['batch']:<7} {r['acc']:<10.2f}%")

    best = results[0]
    avg = sum(r['acc'] for r in results) / len(results)

    print("\n" + "="*70)
    print("Статистика")
    print("="*70)
    print(f"Всего экспериментов: {len(results)}")
    print(f"Лучшая accuracy: {best['acc']:.2f}%")
    print(f"Средняя accuracy: {avg:.2f}%")
    print(f"Разброс: {results[0]['acc'] - results[-1]['acc']:.2f}%")

    print("\n" + "="*70)
    print("Воспроизведение лучшего результата")
    print("="*70)
    print(f"python train_hydra.py \\")
    print(f"    model.hidden_size={best['hidden']} \\")
    print(f"    training.learning_rate={best['lr']} \\")
    print(f"    training.batch_size={best['batch']}")
    print("="*70 + "\n")

if __name__ == "__main__":
    analyze()

Writing analyze_results.py


In [ ]:
!python analyze_results.py


Анализ запуска: /content/multirun/2026-01-25/19-49-38

Топ-5 конфигураций:

Rank   Exp    Hidden   LR         Batch   Accuracy  
------------------------------------------------------------
1      4      256      0.001000   64      96.91     %
2      1      128      0.001000   64      96.28     %
3      3      256      0.000100   64      93.22     %
4      2      128      0.010000   64      92.78     %
5      5      256      0.010000   64      92.50     %

Статистика
Всего экспериментов: 6
Лучшая accuracy: 96.91%
Средняя accuracy: 94.01%
Разброс: 4.56%

Воспроизведение лучшего результата
python train_hydra.py \
    model.hidden_size=256 \
    training.learning_rate=0.001 \
    training.batch_size=64



# Интеграция с другими инструментами

Hydra хорошо интегрируется с другими компонентами ML-стека.

**Фреймворки для обучения**

```python
import hydra
from omegaconf import DictConfig
import pytorch_lightning as pl

@hydra.main(config_path="conf", config_name="config")
def train(cfg: DictConfig):
    model = LightningModel(
        lr=cfg.training.learning_rate,
        hidden_size=cfg.model.hidden_size
    )
    trainer = pl.Trainer(
        max_epochs=cfg.training.epochs,
        accelerator="auto"
    )
    trainer.fit(model)
```

**Системы отслеживания экспериментов**

```python
import hydra
from omegaconf import DictConfig, OmegaConf

@hydra.main(config_path="conf", config_name="config")
def train(cfg: DictConfig):
    # Конфиг легко преобразуется в словарь для логирования
    config_dict = OmegaConf.to_container(cfg, resolve=True)
    
    # Используется с любой системой трекинга
    # experiment.log_parameters(config_dict)
    
    # Обучение модели
    # ...
```

**Преимущества интеграции**

Hydra становится единой точкой управления конфигурациями, которая:
- Предоставляет параметры фреймворку обучения
- Передаёт настройки в систему отслеживания экспериментов
- Сохраняет конфигурации для воспроизводимости
- Поддерживает запуск через системы оркестрации

Это позволяет использовать различные инструменты вместе, сохраняя конфигурации в едином формате.

# Рекомендации по использованию

**Организация конфигов**

```yaml
conf/
  config.yaml           # Базовая конфигурация
  model/
    resnet.yaml
    vgg.yaml
  optimizer/
    adam.yaml
    sgd.yaml
  dataset/
    mnist.yaml
    cifar10.yaml
```


**Версионирование конфигов**

Конфигурационные файлы следует хранить в системе контроля версий вместе с кодом. Это обеспечивает связь между версией кода и настройками экспериментов.

# Заключение

В этом блокноте мы рассмотрели использование Hydra для управления конфигурациями в ML-проектах.

**Основные возможности:**

1. Хранение параметров в YAML файлах вместо хардкода
2. Переопределение параметров из командной строки
3. Композиция конфигов для модульности
4. Автоматический grid search через multirun
5. Сохранение конфигураций для воспроизводимости
6. Интеграция с другими инструментами ML-стека

**Практическая ценность:**

Hydra решает задачи, которые возникают при масштабировании ML-экспериментов:
- Упрощает тестирование множества комбинаций параметров
- Стандартизирует управление конфигурациями в команде
- Обеспечивает модульность и переиспользование настроек
- Интегрируется с фреймворками обучения и системами tracking'а
- Подходит для использования в production системах

**Продвинутые возможности:**

   - [Structured Configs](https://hydra.cc/docs/tutorials/structured_config/intro/)
   - [Optuna Sweeper](https://hydra.cc/docs/plugins/optuna_sweeper/)
   - [Custom Resolvers](https://hydra.cc/docs/advanced/override_grammar/extended/)
